<a href="https://colab.research.google.com/github/GuendouziWassila/KNN_BookRecommendationn/blob/main/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-01-12 01:56:43--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.5’

book-crossings.zip. 100%[===================>]  24.88M  23.2MB/s    in 1.1s    

2024-01-12 01:56:45 (23.2 MB/s) - ‘book-crossings.zip.5’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})


df = df_ratings

counts1 = df['user'].value_counts()
counts2 = df['isbn'].value_counts()

df = df[~df['user'].isin(counts1[counts1 < 200].index)]
df = df[~df['isbn'].isin(counts2[counts2 < 100].index)]

df = pd.merge(right=df, left = df_books, on="isbn")

df = df.drop_duplicates(["title", "user"])

print (df_ratings.shape[0])
print (df_ratings.shape[1])
print (df_ratings.index)


1149780
3
RangeIndex(start=0, stop=1149780, step=1)


In [ ]:
# create the model

pivot_ratings = df.pivot(index='title', columns='user', values='rating').fillna(0)


knn_model = NearestNeighbors(metric='cosine', algorithm='brute')
knn_model.fit(pivot_ratings.values)


NearestNeighbors(algorithm='brute', metric='cosine')

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book_title):

    #Given an item, find its nearest neighbors
    book_isbn = df_books.loc[     df_books['title']==book_title, 'isbn'   ]
    if not book_isbn.empty:
      isbn_to_find = book_isbn.values[0]
      print(f"The ISBN for '{book_title}' is {isbn_to_find}")
    else:
      return(f"No book with the title '{book_title}' found.")

    # Find neighbors

    distances, indices = knn_model.kneighbors(pivot_ratings.loc[book_title].array.reshape(1, -1), n_neighbors=5)

    '''
    recommended_books=[]
    for ind in indices[0]:
      recommended_books.append (pivot_ratings.index[ind])

    distances_list = distances.flatten()
    #print (recommended_books)

    #Combine the book title with its recommended books and distances
    result = [book_title, list(zip(recommended_books[1:], distances_list[1:]))]
    '''
    R_books=[]
    for distance,indice in zip(distances[0],indices[0]):
      if distance!=0:
       R_book=pivot_ratings.index[indice]
       R_books.append([R_book,distance])
    result=[book_title,R_books[::-1]]
    return result


In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")

  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

The ISBN for 'Where the Heart Is (Oprah's Book Club (Paperback))' is 0446672211
["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
The ISBN for 'Where the Heart Is (Oprah's Book Club (Paperback))' is 0446672211
You passed the challenge! 🎉🎉🎉🎉🎉
